# Advent of Code

## 2023-012-025
## 2023 025

https://adventofcode.com/2023/day/25

In [1]:
from collections import defaultdict

def parse_input(filename):
    """ Parse the input file to create a graph. """
    graph = defaultdict(set)
    with open(filename) as f:
        for line in f:
            parts = line.strip().split(":")
            node = parts[0].strip()
            connections = parts[1].strip().split()
            for conn in connections:
                graph[node].add(conn)
                graph[conn].add(node)
    return graph

def dfs(graph, node, visited, low, disc, parent, bridges, time):
    """ Perform DFS to find all bridges in the graph. """
    visited[node] = True
    disc[node] = low[node] = time
    time += 1

    for neighbor in graph[node]:
        if not visited[neighbor]:
            parent[neighbor] = node
            dfs(graph, neighbor, visited, low, disc, parent, bridges, time)
            low[node] = min(low[node], low[neighbor])
            if low[neighbor] > disc[node]:
                bridges.append((node, neighbor))  # A bridge found
        elif neighbor != parent.get(node, None):  # Back edge and not the parent
            low[node] = min(low[node], disc[neighbor])

def find_bridges(graph):
    """ Find all bridges in the graph. """
    visited = {node: False for node in graph}
    disc = {node: -1 for node in graph}  # discovery time
    low = {node: -1 for node in graph}  # lowest reachable node
    parent = {}  # parent of nodes in DFS
    bridges = []
    time = 0
    
    for node in graph:
        if not visited[node]:
            dfs(graph, node, visited, low, disc, parent, bridges, time)
    
    return bridges

def bfs(graph, start, blocked_edge):
    """ Perform BFS to count the number of nodes in the connected component, excluding blocked_edge. """
    visited = set()
    queue = [start]
    visited.add(start)
    
    while queue:
        node = queue.pop(0)
        for neighbor in graph[node]:
            if (node, neighbor) == blocked_edge or (neighbor, node) == blocked_edge:
                continue
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    
    return len(visited)

def solve(filename):
    """ Main function to solve the problem. """
    # Parse the input and create a graph
    graph = parse_input(filename)
    
    # Find all bridges in the graph
    bridges = find_bridges(graph)
    
    # Try disconnecting each combination of three bridges
    max_product = 0
    for i in range(len(bridges)):
        for j in range(i + 1, len(bridges)):
            for k in range(j + 1, len(bridges)):
                # Disconnect the three bridges
                bridge1 = bridges[i]
                bridge2 = bridges[j]
                bridge3 = bridges[k]
                
                # Make a copy of the graph for BFS
                graph_copy = defaultdict(set, {key: set(value) for key, value in graph.items()})
                graph_copy[bridge1[0]].remove(bridge1[1])
                graph_copy[bridge1[1]].remove(bridge1[0])
                graph_copy[bridge2[0]].remove(bridge2[1])
                graph_copy[bridge2[1]].remove(bridge2[0])
                graph_copy[bridge3[0]].remove(bridge3[1])
                graph_copy[bridge3[1]].remove(bridge3[0])
                
                # Use BFS to get the sizes of the two components
                component1_size = bfs(graph_copy, list(graph_copy.keys())[0], bridge1)
                component2_size = bfs(graph_copy, list(graph_copy.keys())[1], bridge1)
                
                # Calculate the product of the two component sizes
                product = component1_size * component2_size
                max_product = max(max_product, product)
    
    return max_product

# Input file name
filename = "input.txt"

# Solve the problem
result = solve(filename)
print("Result:", result)

Result: 0
